# 3D plotter for OptNPath outputs

## Description of the task
OptNPath is a software that enables the computation of an NEB path using various computational chemistry software, including Gaussian.

After running, it delivers a list of files for each optimized path, which contain the xyz coordinates and energies of each point. The aim of this notebook is to render the output of such paths in order to have an interpretable figure, rather than just a list of energies.

In [ ]:
# Imports
%matplotlib widget
import cclib
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
np.set_printoptions(threshold=10)

## Extract data from list of files
Data looks like:
```
     6
 Geometry   1/ 18 for iteration 118 E=    -954.044825715
     C    0.000000    0.000000    0.000000
    CL    1.791689    0.000000    0.000000
     H   -0.358375    1.037571    0.000000
     H   -0.356525   -0.518921   -0.898137
     H   -0.360734   -0.518109    0.897921
    CL   -6.000000    0.001000    0.000000

```

In [ ]:
basedir = Path().cwd()
files_dir = basedir.joinpath("data/optnpath")
optnpath_files = [Path(i) for i in files_dir.glob('test.*')]
optnpath_files = np.array(sorted(optnpath_files, key=lambda x: int(x.suffix[1:])))

In [ ]:
# Take a subset of files:
optnpath_files = optnpath_files[40:80]

n_files = np.size(optnpath_files)

with open(optnpath_files[0]) as sample_file:
    n_steps = sample_file.read().count("Geometry")

print(n_files)
print(n_steps)
print(optnpath_files)

In [ ]:
def get_energies(optnpath_file):
    """
    Extract list of energies in given file
    """
    energies = np.empty(n_steps)
    i = 0
    with open(optnpath_file, "r") as file:
        for line in file.readlines():
            if line.strip().startswith("Geometry"):
                # Geometry   1/ 18 for iteration 118 E=    -954.044825715
                energy = float(line.split("E=")[1].strip())
                energies[i] = energy
                i += 1
    return energies

In [ ]:
energies_table = np.empty((n_files, n_steps))
i = 0
for file in optnpath_files:
    energies = get_energies(file)
    energies_table[i,] = energies
    i += 1
    
print(energies_table)

## Plotting in 3D

In [ ]:
ax = plt.axes(projection='3d')
def f(x,y):
    return energies_table[x,y]

x = np.arange(n_files)
y = np.arange(n_steps)
X, Y = np.meshgrid(y, x)
ax.plot_surface(X, Y, energies_table)